In [ ]:
import json
from langchain.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain

from neo4j import Auth
from botocore.awsrequest import AWSRequest
from botocore.credentials import Credentials
from botocore.auth import (
  SigV4Auth,
  _host_from_url,
)

SCHEME = "basic"
REALM = "realm"
SERVICE_NAME = "neptune-db"
DUMMY_USERNAME = "username"
HTTP_METHOD_HDR = "HttpMethod"
HTTP_METHOD = "GET"
AUTHORIZATION = "Authorization"
X_AMZ_DATE = "X-Amz-Date"
X_AMZ_SECURITY_TOKEN = "X-Amz-Security-Token"
HOST = "Host"


class NeptuneAuthToken(Auth):
  def __init__(
    self,
    credentials: Credentials,
    region: str,
    url: str,
    **parameters
  ):
    # Do NOT add "/opencypher" in the line below if you're using an engine version older than 1.2.0.0
    request = AWSRequest(method=HTTP_METHOD, url=url + "/opencypher")
    request.headers.add_header("Host", _host_from_url(request.url))
    sigv4 = SigV4Auth(credentials, SERVICE_NAME, region)
    sigv4.add_auth(request)

    auth_obj = {
      hdr: request.headers[hdr]
      for hdr in [AUTHORIZATION, X_AMZ_DATE, X_AMZ_SECURITY_TOKEN, HOST]
    }
    auth_obj[HTTP_METHOD_HDR] = request.method
    creds: str = json.dumps(auth_obj)
    super().__init__(SCHEME, DUMMY_USERNAME, creds, REALM, **parameters)

In [8]:
from neo4j import GraphDatabase
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

aws_access_key_id=os.getenv('AWS_KEY_ID'),
aws_secret_access_key=os.getenv('AWS_SECRET_KEY'),
url = os.getenv('AWS_NEPTUNE_URL')
region = os.getenv('AWS_REGION')

# credentials = Credentials(access_key=aws_access_key_id[0], secret_key=aws_secret_access_key[0])
# authToken = NeptuneAuthToken(credentials, region , url)
# driver = GraphDatabase.driver(url, auth=authToken, encrypted=True)

print(url)

driver = GraphDatabase.driver(url, auth=("username", "password"), encrypted=True, trust='TRUST_ALL_CERTIFICATES')

bolt://neptune.geoprism.net:8182/opencypher


In [9]:

# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

from classes import Neo4jConnection
from rdflib import Dataset
from rdflib.namespace import RDF

conn = Neo4jConnection(driver)

try:
    res = conn.query("MATCH (n) RETURN n LIMIT 1")

    # # Clean the database
    # # res = conn.query(f"match (a) -[r] -> () delete a, r")
    # # res = conn.query(f"match (a) delete a")
    # # res = conn.query(f"DROP INDEX entities IF EXISTS")
    # # res = conn.query(f"DROP INDEX entity_index IF EXISTS")

    # dataset = Dataset()
    # dataset.parse("data/model.trig", "format=trig")

    # # @prefix geo: <http://www.opengis.net/ont/geosparql#> .
    # # @prefix lpgv: <https://dev-georegistry.geoprism.net/lpg/deliverable2024/0#> .
    # # @prefix lpg: <https://dev-georegistry.geoprism.net/lpg#> .
    # # @prefix lpgs: <https://dev-georegistry.geoprism.net/lpg/rdfs#> .
    # # @prefix sf: <http://www.opengis.net/ont/sf#> .
    # # @prefix lpgvs: <https://dev-georegistry.geoprism.net/lpg/deliverable2024/0/rdfs#> .
    # # @prefix dcterms: <http://purl.org/dc/terms/> .
    # # @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

    # school_query = """
    # PREFIX lpgvs: <https://dev-georegistry.geoprism.net/lpg/deliverable2024/0/rdfs#>
    # PREFIX lpgs: <https://dev-georegistry.geoprism.net/lpg/rdfs#>
    # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    # SELECT ?schoolCode ?schoolLabel ?zoneCode ?zoneLabel ?areaCode ?areaLabel ?leveeCode ?leveeLabel ?channelCode ?channelLabel
    # WHERE {
    #     ?school a lpgvs:School .
    #     ?school lpgs:GeoObject-code ?schoolCode .
    #     ?school rdfs:label ?schoolLabel .   
    #     ?school lpgvs:HasSchoolZone ?zone .
    #     ?school lpgvs:HasFloodRisk ?area .    
    #     ?area lpgvs:HasFloodZone ?levee .
    #     ?levee lpgvs:HasFloodZone ?channel .        
    #     ?zone lpgs:GeoObject-code ?zoneCode .
    #     ?zone rdfs:label ?zoneLabel .      
    #     ?area lpgs:GeoObject-code ?areaCode .
    #     ?area rdfs:label ?areaLabel .   
    #     ?levee lpgs:GeoObject-code ?leveeCode .
    #     ?levee rdfs:label ?leveeLabel .           
    #     ?channel lpgs:GeoObject-code ?channelCode .
    #     ?channel rdfs:label ?channelLabel .           
    # }
    # LIMIT 1
    # """

    # for c in dataset.graphs():  # doctest: +SKIP
    #     if 'deliverable2024' in str(c.identifier):
        
    #         qres = c.query(school_query)
    
    #         for row in qres:
                
    #             schoolLabel = row.schoolLabel.replace("'", "\\'")
    #             zoneLabel = row.zoneLabel.replace("'", "\\'")
    #             areaLabel = row.areaLabel.replace("'", "\\'")
    #             leveeLabel = row.leveeLabel.replace("'", "\\'")
    #             channelLabel = row.channelLabel.replace("'", "\\'")
            
    #             statement = (
    #                 f"MERGE (area:Entity:FloodArea {{name: '{areaLabel}', code: '{row.areaCode}'}})"
    #                 f"MERGE (school:Entity:School {{name: '{schoolLabel}', code: '{row.schoolCode}'}})"
    #                 f"MERGE (zone:Entity:SchoolZone {{name: '{zoneLabel}', code: '{row.zoneCode}'}})"                
    #                 f"MERGE (levee:Entity:Levee {{name: '{leveeLabel}', code: '{row.leveeCode}'}})"                                    
    #                 f"MERGE (channel:Entity:Channel {{name: '{channelLabel}', code: '{row.channelCode}'}})"                                                        
    #                 f"MERGE (zone)-[:HAS_SCHOOL]->(school)"
    #                 f"MERGE (school)-[:HAS_FLOOD_AREA]->(area)"                
    #                 f"MERGE (area)-[:PROTECTED_BY]->(levee)"                                    
    #                 f"MERGE (levee)-[:PROTECTS_AGAINST]->(channel)"                                                        
    #             )
                
    #             print(statement)
            
    # #             res = conn.query(statement)
            
    # # conn.query("CREATE FULLTEXT INDEX entities FOR (n:Entity) ON EACH [n.name]")
finally:
    print('done')
    # conn.close()       

Query failed: Couldn't connect to neptune.geoprism.net:8182 (resolved to ()):
Connection to 44.225.75.201:8182 closed without handshake response
done
